welcome to the pyramco demo notebook. put your api key in the box below. leave the url as-is. click "kernel" above and change it to 3.6, then restart it.

In [ ]:
ramco_api_key = 'your_ramco_api_key_goes_here'
ramco_api_url = 'https://api.ramcoams.com/api/v2/'

run the cell above (which sets your variables to memory,) then run the cell below to prove your variables are set properly by printing them out.

In [ ]:
print(ramco_api_key)
print(ramco_api_url)

the result you're looking for above is:

    your_ramco_api_key_goes_here (or whatever you set your api key to above, hopefully!)
    https://api.ramcoams.com/api/v2/

those two variables will be put in a config file later, but for now this is good enough.
you can also just explicitly state those variables at the beginning of every script, like: 

    ramco_api_key = 'your_ramco_api_key_goes_here'
    ramco_api_url = 'https://api.ramcoams.com/api/v2/'

In [ ]:
### test pyramco setup

# first we import pyramco, which should be in the right place already
import pyramco

# set the variable 'reply' to the value of the function 'pyramco.clear_cache'
# setting the variable implicitly fires the function
reply = pyramco.clear_cache()

# then print the content of the variable 'reply' to screen
print(reply)

assuming all is well, you should see the response:
    
    {'ResponseCode': 204, 'ResponseText': 'Cache cleared.'}

congratulations, you've just written a script that did something useful!

 * GetEntityTypes

 * Get all entity types from ramco.

In [ ]:
# from now on, we import pyramco like this in every script
import pyramco

# set the variable 'types' to the result of running the 
# 'get_entity_types' function

types = pyramco.get_entity_types()

# then print the result
print(types)

 * GetEntityMetadata
 
 * Get metadata for the contact entity in ramco.

In [ ]:
import pyramco

# set the variable 'metadata' to the result of running the 
# 'get_entity_metadata' function on the entity 'Contact'

metadata = pyramco.get_entity_metadata('Contact')

# then print the result
print(metadata)

 * GetOptionSet
 
 * Get a list of valid value/label pairs for the cobalt_PreferredPhone OptionSet on the contact entity.

(using 'ramco_preferredphone' below as cobalt_PreferredPhone doesn't exist in ramco)

In [ ]:
import pyramco

option_set = pyramco.get_option_set('Contact','ramco_preferredphone')

print(option_set)

 * GetEntities - Basic

 * Request data for 10 contact records. No filter statement used yet

In [ ]:
import pyramco

contacts = pyramco.get_entities('Contact', ('ContactId,FirstName,LastName,ramco_nrdsid,ModifiedOn'), max_results=10)

print(contacts)

 * GetEntities - Basic Filtering

 * Request data for max of 10 contact records modified since Jan 1, 2015.
 * When this works...
 
    ResponseCode = 200
    ResponseText = 'OK'
    Data = collection of value/label pairs for the optionset

In [ ]:
import pyramco

contacts = pyramco.get_entities('Contact', ('ContactId,FirstName,LastName,ramco_nrdsid,ModifiedOn'),\
                                max_results=10,filters='ModifiedOn<ge>2015-01-01')

print(contacts)

 * GetEntities - Advanced Filtering

 * Request data for max of 10 contact records modified since Jan 1, 2015 where LastName starts with 'Add' or LastName equals 'Acosta'

In [ ]:
import pyramco

contacts = pyramco.get_entities('Contact', ('ContactId,FirstName,LastName,ramco_nrdsid,ModifiedOn'),\
                                max_results=10,\
                                filters='ModifiedOn<ge>2015-01-01 \
                                AND (LastName<sb>#Add# \
                                OR LastName<eq>#Acosta#)')

print(contacts)

 * GetEntity - Basic

 * Requesting a specific contact record be returned.

In [ ]:
import pyramco

guid = 'get-yourself-a-member-guid'

contact = pyramco.get_entity('Contact', guid,\
                              ('ContactId,Address3_Latitude,FirstName,LastName,'\
                              'ramco_nrdsid,StatusCode,ModifiedOn'))

print(contact)

 * GetEntity - Request with data from relationship

 * Request data for a specific contact record and include data from memberships.
 
 * You can include data from up to six distinct relationships.

In [ ]:
import pyramco

guid = 'get-yourself-a-member-guid'

contact = pyramco.get_entity('Contact', guid,
                              ('ContactId,FirstName,LastName,ramco_nrdsid,'\
                              'StatusCode,ModifiedOn,cobalt_contact_cobalt_membership/StatusCode,'\
                              'cobalt_contact_cobalt_membership/cobalt_AssociationId,'\
                              'cobalt_contact_cobalt_membership/cobalt_MemberTypeId'))
print(contact)

 * UpdateEntity

 * Note that update entity should return:
    ResponseCode = 204
    ResponseText = 'OK - No Data'
    
 * Here we demonstrate a method to geocode a member address and use update entity to push it back into RAMCO

In [ ]:
import googlemaps
import pyramco
from pprint import pprint

# google maps API key
gmaps = googlemaps.Client(key='get-a-key-https://cloud.google.com/maps-platform/')

# your contact's GUID
guid = 'get-yourself-a-member-guid'

# get your contact's full address string
reply_address = pyramco.get_entity('Contact', guid, 'Address1_Composite' )
raw_address = reply_address['Data']['Address1_Composite']
print('Raw Address: ',raw_address)
print('--------------------')

# pass the raw address to google maps api and fetch the values we need
geocode_result = gmaps.geocode(raw_address)
print('Google Maps API return: ',geocode_result)
print('--------------------')

# iterate over the return to get latitude/longitude values
lat = geocode_result[0]['geometry']['location']['lat']
lng = geocode_result[0]['geometry']['location']['lng']
formatted_address = geocode_result[0]['formatted_address']
place_id = geocode_result[0]['place_id']
print('Latitude: ',lat)
print('Longitude: ',lng)
print('Formatted Address: ',formatted_address)
print('Google Place ID: ',place_id)
print('--------------------')

# call the ramco API again to update the record and print the response
update_reply = pyramco.update_entity('Contact', guid, (f'Address3_Name=#{place_id}#,Address3_Composite=#{formatted_address}#,Address3_Latitude=#{lat}#,Address3_Longitude=#{lng}#'), string_delimiter='#')
pprint(update_reply)
